# Reappraisal Training on PyTorch Lightning

## Setup
- When running on Google Colab, mount Google Drive to access scripts.
- `cd` into the project root and install dependencies:

In [ ]:
%load_ext autoreload
%load_ext tensorboard
# from google.colab import drive
# drive.mount('/content/drive')
# %cd {root_dir}

# %pip install pytorch-lightning "ray[tune]" wandb transformers datasets nltk nbdev 
# ! nbdev_install_git_hooks

import nltk
nltk.download('punkt')

ROOT_DIR = '/root/reappraisal-model'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import torch

# Define constants
STRAT = 'obj'
BATCH_SIZE = 64 
TEST_BATCH_SIZE = 64
NUM_FOLDS = 5

## Load LDH Data

Contains the following:

- LDHI
- LDHII
- LDHIII

In [ ]:
%autoreload
from reappraisalmodel.ldhdata import LDHDataModule
ldhdata = LDHDataModule(data_dir=ROOT_DIR, batch_size=BATCH_SIZE, strat=STRAT, kfolds=NUM_FOLDS)

Training data loaded from disk.
Encoding Train Data:



Evaluation data loaded from disk.
Encoding Test Data:


## Run K-Fold Training


In [ ]:
%autoreload 2
from reappraisalmodel.trainers import kfold_train


#TODO: ADD CSVLOGGER!!!!
results = kfold_train(NUM_FOLDS, ldhdata, strat=STRAT, 
                       max_epochs=15, 
#                        limit_train_batches=2,
#                        limit_val_batches=1
                    )


Created temporary directory: /tmp/tmpvsrtlvfl


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training on split 0


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory lightning_logs/reappmodel_obj_20210227_203013/0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory lightning_logs/reappmodel_obj_20210227_203013/3 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training on split 4


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory lightning_logs/reappmodel_obj_20210227_203013/4 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)



Successful 0_epoch=06-val_loss=1.13.ckpt to s3: True
{'val_loss': 1.2094045877456665, 'train_loss': 1.13628089427948, 'num_epochs': 9, 'r2score': tensor(0.5926), 'explained_var': tensor(0.6366)}
Successful 1_epoch=07-val_loss=1.17.ckpt to s3: True
{'val_loss': 1.1819031238555908, 'train_loss': 1.0930699110031128, 'num_epochs': 8, 'r2score': tensor(0.6087), 'explained_var': tensor(0.6377)}
Successful 2_epoch=07-val_loss=1.12.ckpt to s3: True
{'val_loss': 1.2881639003753662, 'train_loss': 1.1193828582763672, 'num_epochs': 10, 'r2score': tensor(0.5832), 'explained_var': tensor(0.6109)}
Successful 3_epoch=04-val_loss=1.15.ckpt to s3: True
{'val_loss': 1.1841994524002075, 'train_loss': 1.1642210483551025, 'num_epochs': 7, 'r2score': tensor(0.6130), 'explained_var': tensor(0.6388)}
Successful 4_epoch=07-val_loss=1.20.ckpt to s3: True
{'val_loss': 1.2711799144744873, 'train_loss': 1.084243655204773, 'num_epochs': 10, 'r2score': tensor(0.5906), 'explained_var': tensor(0.6339)}


In [ ]:
import pandas as pd

pd.DataFrame(results)

,val_loss,train_loss,num_epochs,r2score,explained_var
0,1.209405,1.136281,9,tensor(0.5926),tensor(0.6366)
1,1.181903,1.093070,8,tensor(0.6087),tensor(0.6377)
2,1.288164,1.119383,10,tensor(0.5832),tensor(0.6109)
3,1.184199,1.164221,7,tensor(0.6130),tensor(0.6388)
4,1.271180,1.084244,10,tensor(0.5906),tensor(0.6339)


0.768505042273634

## Tuning Hyperparameters


In [ ]:
%autoreload
from reappraisalmodel.trainers import run_tune
run_tune(ldhdata)

# Exploring

## Shape of Model

In [ ]:
pretrained_model = "distilbert-base-uncased-finetuned-sst-2-english"

from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained(pretrained_model)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

### Tokenization

In [ ]:
# hide
# Returns a BatchEncoding of the text.
tokenized = tokenizer(text = ["This is the first test sentence!", "This is the second, better test sentence."], 
    padding='max_length', max_length=150)

for idx, sent in enumerate(tokenized.input_ids):
    print(f"Sentence            {idx}: {tokenizer.convert_ids_to_tokens(sent)}")
    print(f"Tokenized Attention {idx}: {tokenized[idx].attention_mask}")



### Training

In [ ]:
%autoreload
import torch
import pytorch_lightning as lit
from reappraisalmodel.lightningreapp import LightningReapp

default_config = default_config = {
    'lr': 1e-3,
    'hidden_layer_size': 50
}

model = LightningReapp(default_config)

trainer = lit.Trainer(
    gpus = 1 if torch.cuda.is_available() else None,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    max_epochs=10,
    fast_dev_run=2,
    terminate_on_nan=True)

model = LightningReapp(default_config)

trainer.fit(model, ldhdata.train_dataloader(), ldhdata.val_dataloader())



In [ ]:
model = LightningReapp.load_from_checkpoint(
    '/Users/danielpham/Google Drive/ldh/lightning_logs_obj_0223/version_2/checkpoints/epoch=1-step=337.ckpt', map_location='cpu')

model.eval()
model

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer 

model.eval()
trainer = Trainer(
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    terminate_on_nan=True)

test_dataloader = ldhdata

In [ ]:
dfs = []
for result in results:
    print(len(result['predict']))

In [ ]:
import pickle
with open('./output_reapp.pkl', 'rb+') as f:
    results = pickle.load(f)

In [ ]:
import boto3
import pickle

from sagemaker import get_execution_role
role = get_execution_role()

bucket = 'ldhdata'
file = 'Master_Final_TrainingData.csv'

s3client = boto3.client('s3')

response = s3client.get_object(Bucket=bucket, Key=file)

import codecs 
import csv

train = csv.DictReader(codecs.getreader("utf-8")(response["Body"])) # returns an ordered dict


In [ ]:
import torch
from pytorch_lightning.metrics.functional import r2score, explained_variance

expected = torch.rand(16)
observed = torch.rand(16)

r2score(expected, observed)

In [ ]:
!nbdev_update_lib